In [18]:
import geopandas
import re
import pandas as pd
import folium
import requests
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [19]:
# API Call Function

def call_api(url, lta_api_key):
    
    # Define endpoint to call
    endpoint = url

    # Define the headers, including the API key
    headers = {
        'accept': 'application/json',
        'AccountKey': lta_api_key
    }

    # Initialise the output list to hold the results
    output = []

    # Loop through the pages to get all the results
    while (len(output) % 500) == 0:

        # Set the params to skip to the correct page
        params = {"$skip": len(output)}

        # Make the GET request
        req = requests.get(endpoint, headers = headers, params = params)

        # Extend the output list to include the results
        output.extend(req.json()['value'])

    return pd.DataFrame(output)



In [20]:
# API Key

api_key = 'your_API_key'

In [21]:
# Bus Services

bus_services = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusServices", api_key)
bus_services

,ServiceNo,Operator,Direction,Category,OriginCode,DestinationCode,AM_Peak_Freq,AM_Offpeak_Freq,PM_Peak_Freq,PM_Offpeak_Freq,LoopDesc
0,118,GAS,1,TRUNK,65009,97009,5-08,8-12,8-10,09-14,
1,118,GAS,2,TRUNK,97009,65009,10-10,8-11,4-08,9-12,
2,118A,GAS,1,TRUNK,65009,96119,06-66,-,-,-,
3,118B,GAS,1,TRUNK,96111,65191,-,-,24-57,-,
4,119,GAS,1,TRUNK,65009,65009,09-13,12-18,12-15,15-17,Hougang St 21
...,...,...,...,...,...,...,...,...,...,...,...
722,98B,TTS,1,TRUNK,28501,21099,05-52,-,-,-,
723,98M,TTS,1,TRUNK,28009,28009,-,17-18,-,12-17,Corporation Rd
724,990,TTS,1,TRUNK,43009,43009,11-11,13-15,12-14,12-14,Jurong Gateway Rd
725,992,TTS,1,TRUNK,43009,41009,03-09,08-12,04-08,04-13,


In [22]:
# Bus Routes

bus_routes = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes", api_key)
bus_routes

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
25543,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-
25544,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-
25545,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-
25546,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-


In [23]:
# Bus Stops

bus_stops = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusStops", api_key)
bus_stops

,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5135,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5136,99161,Nicoll Dr,Aft Changi Beach CP 3,1.390262,103.992957
5137,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5138,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [24]:
# Bus Routes with Bus Stop Descriptions

bus_route_names = pd.merge(bus_routes, bus_stops, on = 'BusStopCode', how = 'left')
bus_route_names

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,Tampines Ave 5,Blk 147,1.348753,103.942086
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,Simei Ave,Changi General Hosp,1.340055,103.948381
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,Simei Ave,Opp Blk 3012,1.337371,103.950673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25543,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-,Nicoll Dr,SAF Ferry Ter,1.387214,103.999714
25544,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-,Airline Rd,Police Pass Off,1.385252,103.999834
25545,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-,Airline Rd,Aft Cargo Bldg D,1.381386,103.998191
25546,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-,Airline Rd,Airline Hse,1.377552,103.996539


In [25]:
# Bus Stop Codes & Coordinates

bus_stops_coordinates = bus_stops[['BusStopCode', 'Latitude', 'Longitude']]
bus_stops_coordinates['BusStopCode'] = bus_stops_coordinates['BusStopCode'].astype(int)

bus_stops_coordinates

,BusStopCode,Latitude,Longitude
0,1012,1.296848,103.852536
1,1013,1.297710,103.853225
2,1019,1.296990,103.853022
3,1029,1.296673,103.854414
4,1039,1.298208,103.855491
...,...,...,...
5135,99139,1.388195,103.987234
5136,99161,1.390262,103.992957
5137,99171,1.391128,103.991021
5138,99181,1.387754,103.988503
